# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [51]:
import statsmodels.api as sm
import pandas as pd

In [3]:
# Import the data

df = pd.read_csv('titanic.csv', index_col=0)


In [40]:
df.shape

(891, 11)

## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [4]:
# Total number of people who survived/didn't survive
df[df['Survived'] == 1]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C
880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S


In [35]:
df.Pclass.value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [54]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'Fare', 'Sex', 'Embarked', 'Survived']
rel_df = df[relevant_columns]
dummy_dataframe = pd.get_dummies(rel_df, columns=['Pclass', 'Sex', 'Embarked'], drop_first=True)

dummy_dataframe.shape

(891, 8)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [55]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(714, 8)

In [56]:
dummy_dataframe.head()

,Age,Fare,Survived,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
PassengerId,,,,,,,,
1,22.0,7.2500,0,0,1,1,0,1
2,38.0,71.2833,1,0,0,0,0,0
3,26.0,7.9250,1,0,1,0,0,1
4,35.0,53.1000,1,0,0,0,0,1
5,35.0,8.0500,0,0,1,1,0,1


Finally, assign the independent variables to `X` and the target variable to `y`: 

In [57]:
# Split the data into X and y
y = dummy_dataframe['Survived']
X = dummy_dataframe.drop('Survived', axis = 1)

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [58]:
# Build a logistic regression model using statsmodels
X = sm.add_constant(X)

# Fit model
logit_model = sm.Logit(y, X)

# Get results of the fit
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.450357
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [60]:
# Summary table
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      706
Method:                           MLE   Df Model:                            7
Date:                Wed, 01 Apr 2020   Pseudo R-squ.:                  0.3332
Time:                        21:55:24   Log-Likelihood:                -321.55
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.613e-65
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0644      0.505      8.052      0.000       3.075       5.054
Age           -0.0360      0.008     -4.624      0.000      -0.051      -0.021
Fare          -0.0001      0.002     -0.061      0.951      -0.005       0.004
Pclass_2      -1.1565      0.320     -3.613      0.000      -1.784      -0.529
Pclass_3      -2.4232      0.333     -7.277      0.000      -3.076      -1.771
Sex_male      -2.5239      0.210    -12.018      0.000      -2.936      -2.112
Embarked_Q    -0.8269      0.570     -1.452      0.147      -1.943       0.289
Embarked_S    -0.5057      0.269     -1.878      0.060      -1.033       0.022
==============================================================================
"""

In [ ]:
# Your comments here
'''
age, passenger class, sex, are relevant
fare, and embarked aren't
'''

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [ ]:
# Your code here


In [ ]:
# Your comments here

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!